# TME 8 : Mini-projet Detection de motifs


## Localisation de pattern (motifs) dans le genome

Nous allons implémenter plusieurs algorithmes afin de localiser un motif d'intérêt dans un génome.

### A -  Boyer Moore Algorithm 
1\. Nous allons réutiliser les fonctions du TME6 pour générer une séquence artificiel de taille ``n``, et implanter un motif de taille ``k`` dans une position aléatoire de la séquence. Vous pouvez aussi faire varier ``v`` positions pour permettre certaines mutations


In [22]:
import random
import numpy as np
import time

nuc = ('A', 'C', 'G', 'T')

t = 1
n = 100 #longuer des sequence
k = 8 #longuer du motif
v = 0 

def generateRandomSequences(n, t):
    sequences = []
    i = 0
    j = 0
    while i < t:
        seq = ""
        while j < n:
            seq = seq+nuc[random.randint(0,3)]
            j += 1
        sequences.append(seq)
        i += 1
        j = 0
   
    return sequences

# RENVOIE UNE LISTE DE LISTE DE MOTIFS...
def generateMotifs(k, nM):
    return [generateRandomSequences(k, True) for rs in range(nM)]

def gen_n_substitMotifs(v, k, t):
    motifs = generateMotifs(k, 1)
    print(motifs[0])
    motif = ''.join(motifs[0])
    motifs = []
    i=0
    while i < t:
        j = 0
        motif_a_modif = list(motif.strip())
        while j < v:
            motif_a_modif[random.randint(0, k - 1)] = nuc[random.randint(0,3)]
            motif_add = ''.join(motif_a_modif)
            j+=1
        motifs.append(motif_add)
        i+=1
    return motifs
    
def implantMotifVar(k, v, t, n):
    #sequences aleatoires
    sequences = generateRandomSequences(n, t)
    if v != 0:
        motifs = gen_n_substitMotifs(v, k, t)
    else:
        motifs_generated = generateMotifs(k, 1)
        # il faut remettre dans une simple liste vu QUE GENERATE_MOTIFS fait une liste de LISTE
        motifs = motifs_generated[0]
    modified_sequences = []
    i = 0
    while i < t:
        seq = sequences[i]
        if v != 0:
            motif = motifs[i]
        else:
            motif = motifs[0]
        rand_pos = random.randint(0,len(seq))
        seq = seq[0:rand_pos] + motif + seq[rand_pos+1:len(seq)]
        modified_sequences.append(seq)
        i+=1
    return motifs, modified_sequences

In [23]:
#generate and implant variavel motifs
motifs, sequences = implantMotifVar(k, v, t, n)
motif = motifs[0]
sequence = sequences[0]
print("motif : {}".format(motif))
print("sequence : {}".format(sequence))

motif : TGGCCTAC
sequence : CTCCCCTGGCCTACGTGTAGTGGCCGCCGATATCTCACACATCTAAGTCACATTTGGTCGGTGCGCACTTGTACCAAAAAAATTCGAGCCCCTTGCCCACAGGTCCG


2\. Boyer Moore est une combinaison de deux approches : bad character et good Suffix. Nous allons commencer par la première approche. Définissez une fonction ``badCharacter`` pour construire la badTable d'un motif. Vous pouvez l'implémenter comme dictionnaire, voir un exemple ci-dessous.

<img src="boyer-moore-badTable.png">

In [24]:
def badCharacter(motif):
    '''
    The preprocessing function for Boyer Moore's bad character heuristic
    '''
    badChar = {}
    i = 0
    while i < len(motif):
        char = motif[i]
        badChar[char] = max(1, len(motif) - i - 1)
        i+=1
    
    #print(badChar)
    return badChar
           
badCharacter("TATGTG") #{'T': 1, 'A': 4, 'G': 1}
badCharacter(motif)

{'T': 2, 'G': 5, 'C': 1, 'A': 1}

3\. Développer la version de l'algorithme Boyer Moore qui utilise "bad character" la règle de caractère incorrect.

In [25]:
import sys
def search(genome, motif):
    '''
    A pattern searching function that uses Bad Character Heuristic of Boyer Moore Algorithm
    '''
    bc = badCharacter(motif)
    pos = 0
    i = 0
    cpt = 0
    while pos + (len(motif) - i) < len(genome):
        cpt += 1
        char_gen = genome[pos + (len(motif) - i) - 1]
        char_mot = motif[len(motif) - i - 1]
        if char_gen != char_mot:
            if char_gen not in bc:
                pos += len(motif)
            else:
                pos += bc[char_gen]
            i = 0
        else:
            i += 1
        if i >= len(motif):
            break
    
    #print("Nombre de tours de boucle : ",cpt)
    #print(pos)
    if pos + (len(motif) - i) >= len(genome):
        return -1

    return pos

print(search("ATCGCA", "TA")) #-1
            
print(search("ATGTCCGTC", "CCG")) #4

print(search("ATCGCA", "GC")) #3

print(search("CGTGGCCAAGGAGCT", "GTGGCCAA"))

print(search(sequence, motif))

-1
4
3
-1
6


4\. Testez la version de l'exercice 3 sur les données artificielles généré dans 1, combien de comparaison inutile avez-vous évité avec cette version? 

In [26]:
print(motif)
print(sequence)
pos = search(sequence, motif)
print(pos)
print(sequence[pos:pos + k])
sequence.find(motif)

TGGCCTAC
CTCCCCTGGCCTACGTGTAGTGGCCGCCGATATCTCACACATCTAAGTCACATTTGGTCGGTGCGCACTTGTACCAAAAAAATTCGAGCCCCTTGCCCACAGGTCCG
6
TGGCCTAC


6

Reponse: On obtient le bon motif. Nous aurions fait 92 comparaisons normalement, (100(taille sequence) - 8(taille motif)), ici nous en avons fait 24 nous en avons donc eviter 68.

5\. Nous allons implémenter la règle de good suffixe. Pour cela, nous allons pré-processer le motif afin de trouver tous les suffixe de taille k que sont aussi de préfixe. Faire la fonction goodSuffix qui étant donnée un motif renvoie un dictionnaire ou les clés sont de suffixes et les valeurs sont la distance au préfixe le plus proche, voir un exemple ci-dessous. d2 est la distance entre le suffixe de taille k et le son occurence la plus proche.

<img src="suffix.png">

In [27]:

def goodSuffix(motif):
    gs={}
    i=0
    suffixe=""
    tmplength=0
    while i<len(motif)-1:
        suffixe=motif[len(motif)-i-1]+suffixe
        if i!=len(motif)-1:
            char_1=motif[len(motif)-i-2]
        j=0
        """
        on essaie de trouver le suffixe dans le reste du motif
        """
        while j+i<len(motif)-1:
            if suffixe==motif[j:j+i+1]:
                if j!=0 and i!=len(motif)-1:
                    if char_1!=motif[j-1]:
                        gs[suffixe]=len(motif)-i-j-1
                else :
                    gs[suffixe]=len(motif)-i-j-1
            j+=1
        i+=1
        
        """
        cas échéant : on essaie de voir si la fin du suffixe matche avec le préfixe, sinon on lui associe la longueur du motif
        """
        maxl=0
        if suffixe not in gs:
            for s in gs:
                if s == motif[0:len(s)] and maxl<len(s):
                    maxl=len(s)
            if maxl!=0:
                gs[suffixe]=len(motif)-maxl
            else:
                gs[suffixe]=len(motif)
    return gs

gs = goodSuffix("ABCBAB"); print (gs)  #{'B': 2, 'AB': 4, 'BAB': 4, 'CBAB': 4, 'BCBAB': 4}
gs = goodSuffix("BAOBAB"); print (gs)  #{'B': 2, 'AB': 2, 'BAB': 5, 'OBAB': 5, 'AOBAB': 5}


{'B': 2, 'AB': 4, 'BAB': 4, 'CBAB': 4, 'BCBAB': 4}
{'B': 2, 'AB': 5, 'BAB': 5, 'OBAB': 5, 'AOBAB': 5}


6\. Développer la version de l'algorithme Boyer Moore qui utilise les deux règles "bad character" et  "good suffixe". Avez-vous amélioré votre algorithme? 

In [52]:
import sys
def search(genome, motif):
    '''
    A pattern searching function that uses Bad Character
    Heuristic of Boyer Moore Algorithm
    '''
    start = time.time()
    bc = badCharacter(motif) 
    gs = goodSuffix(motif)
    
    res = -1
    cpt = 0
    pos = 0
    offset = 1
    while pos < len(genome) - len(motif):
        cpt += 1
        char_genome = genome[pos + len(motif) - offset]
        char_motif = motif[len(motif) - offset]
        if char_genome != char_motif:
            suffixe_genome = genome[pos + len(motif) - offset:pos + len(motif)]
            badchar = bc[char_genome]
            if suffixe_genome in gs:
                goodsuff = gs[suffixe_genome]
            else:
                goodsuff = -1
            pos += max(badchar, goodsuff)
            offset = 1
        else:
            offset += 1
        if offset > len(motif):
            res = pos
            break

    
    return res
              
#search("ACAAABABABC", "ABABAB")
print(search("GGACGGCGTGGCCAAGGAGCT", "GTGGCCAA"))
start = time.time()
print(search(sequence, motif))
end = time.time()
print(end - start)
       

-1
6
0.0005009174346923828


In [21]:
#Reponse : Sur cette sequence l''algoritme ne montre pas d''amelioration

7\. Tester l'algorithme **Boyer Moore Algorithm** sur vos données de chipSeq. Vous pouvez chercher de motifs trouvés par les algorithms développé précédemment

In [29]:
def readFasta(genome):
    sequence = []
    file = open(genome, "r")
    sequences = []
    seq = ""
    for s in file:
        if s[0] != ">":
            seq += s.strip().upper()
        else:
            sequences.append(seq)
            seq = ""
    return sequences[1:]
genome = "Sequence_by_Peaks_6.fasta"
sequences = readFasta(genome)
#print(sequences)

In [55]:
start = time.time()
liste_res = []
for seq in sequences:
    liste_res.append(search(seq, "TTTGCTCA"))
    #print("TTTGCTCA :   ",search(seq, "TTTGCTCA"))
end = time.time()
print(end - start)
print(liste_res)

0.011501789093017578
[-1, 180, -1, -1, -1, -1, -1, -1, 370, -1, -1, -1, 489, 197, -1, -1, -1, -1, -1, 155, 507, -1, -1, 373, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


### B -  Index of fixed length words
8\. Faire une fonction pour indexer les positions d'occurrences de tous les mots de taille k dans un texte, la fonction doit renvoyer un dictionnaire ou les clés sont les mots et les valeurs les positions dans le texte.

In [56]:
def indexTable(k, texte):
    indexes  = {}

    i = 0
    while i < len(texte) - k + 1:
        mot = texte[i:i + k]
        if mot not in indexes:
            indexes[mot] = [i]
        else:
            indexes[mot].append(i)
        i += 1
            
    return indexes

indexTable(3, "ACTACTACTACT")


{'ACT': [0, 3, 6, 9], 'CTA': [1, 4, 7], 'TAC': [2, 5, 8]}

9\. Faire une fonction pour découper un motif en sous-parties de taille k

In [57]:
def seeds(k, motif):
    sous_motifs = []
    if k > len(motif):
        print("Erreur seeds : K est plus grand que le motif")
        return -1
    i = 0
    while i < len(motif) - k + 1:
        sous_motifs.append(motif[i:i+k])
        i += 1
    return sous_motifs


seeds(3, "ATCATCAGT")
   

['ATC', 'TCA', 'CAT', 'ATC', 'TCA', 'CAG', 'AGT']

10\. Faire une fonction pour chercher un motif dans un texte utilisant la table des indexes de la question 7

In [58]:
import time

def searchIndexTable(texte, k, motif):
    #start = time.time()
    sous_motifs = seeds(k, motif)
    index_table = indexTable(k, texte)
    cpt = 0
    res = -1
    i = 0
    while i < len(sous_motifs):
        sousM = sous_motifs[i]
        if sousM in index_table:
            cpt += 1
            indexes = index_table[sousM]
            for index in indexes:
                genome = texte[index - i: index + k]
                if genome == motif:
                    res = index - i
                    break
        if res != -1:
            break       
        i += 1
    #print("cpt :",cpt)
    #end = time.time()
    #print(end - start)
    return res

k = 2 
start = time.time()
res = searchIndexTable(sequence, k, motif)
end=time.time()
print(end - start)
print(res)


0.0
6


11\. Tester l'algorithme **Index of fixed length** words sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [59]:
liste_res = []
start = time.time()
for seq in sequences:
    liste_res.append(searchIndexTable(seq, 2, "TTTGCTCA"))
    #print("TTTGCTCA :   ",searchIndexTable(seq, 2, "TTTGCTCA"))
end = time.time()
print(end - start)
print(lis)

TTTGCTCA :    -1
TTTGCTCA :    180
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    370
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    489
TTTGCTCA :    197
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    155
TTTGCTCA :    507
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    373
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTCA :    -1
TTTGCTC

### C -  Matrice de fréquences
12\.Nous allons implémenter la recherche de motif par matrice de fréquence. Nous allons utiliser les matrices déjà fabriqué que vous pouvez telecharcher du site http://jaspar.genereg.net/
Une fois que vous avez chargé votre matrice de fréquence, vous devez la transformer en matrice de probabilité

In [165]:
import numpy as np

nuc = ('A', 'C', 'G', 'T')
k = 8

matrice = np.array([[462.00,24.00,0.00,1000.00,0.00,7.00,90.00,600.00],
                    [86.00,16.00,22.00,0.00,1000.00,0.00,198.00,138.00],
                    [387.00,0.00,803.00,0.00,0.00,993.00,0.00,162.00],
                    [65.00,960.00,175.00,0.00,0.00,0.00,712.00,100.00]])

#Computing wi(a)
def pwm(P, k):
    res=[]
    i=0
    j=0
    while i<4:  #Il y aura toujours que 4 bases (A,C,G,T) 
        tempres=[]
        while j<k: #k etant la longueur des motifs, c'est aussi la longueur des profils
            tempres.append(P[i][j]/(P[0][j]+P[1][j]+P[2][j]+P[3][j]))
            j+=1
        j=0
        i+=1
        res.append(tempres)
    return res


PWM = pwm(matrice, k); print (PWM)


[[0.462, 0.024, 0.0, 1.0, 0.0, 0.007, 0.09, 0.6], [0.086, 0.016, 0.022, 0.0, 1.0, 0.0, 0.198, 0.138], [0.387, 0.0, 0.803, 0.0, 0.0, 0.993, 0.0, 0.162], [0.065, 0.96, 0.175, 0.0, 0.0, 0.0, 0.712, 0.1]]


12\. Déterminer les paramètres f(0)(b) du modèle nul, où 
\begin{equation}
f^{(0)}(b) = \frac 1L \sum_{i=0}^{L-1} \omega_i(b)\ ,
\end{equation}

In [166]:
def f0_calcule(PWM, k):
	res = {}
	i = 0
	for ligne in PWM:
		somme_ligne = 0
		for colonne in ligne:
			somme_ligne += colonne
		res[nuc[i]] = somme_ligne / k
		i+=1
	
	return res

f_0 = f0_calcule(PWM, k)
print (f_0)

{'A': 0.272875, 'C': 0.1825, 'G': 0.29312499999999997, 'T': 0.2515}


13\.Faites une fonction pour calculer log-rapport de vraisemblancee, d'une sequence de taille k.
\begin{equation}
\label{eq:ll}
\ell(b_0,...,b_{k-1}) = \log_2 \frac {P(b_0,...,b_{k-1} | \omega )
}{P^{(0)}(b_0,...,b_{k-1})}
= \sum_{i=0}^{k-1} \log_2 \frac {\omega_i(b_i)}{f^{(0)}(b_i)}\ .
\end{equation}

In [167]:
#PFM de Mac1, associé au peaks6
Mac1 = np.array([[108.00, 89.00, 0.00, 115.00, 0.00, 0.00, 0.00, 958.00],
                 [0.00, 0.00, 0.00, 0.00, 1714.00, 0.00, 1714.00, 0.00],
                 [0.00, 219.00, 0.00, 1599.00, 0.00, 0.00, 0.00, 655.00],
                 [1632.00, 1451.00, 1714.00, 0.00, 0.00, 1714.00, 0.00, 113.00]])

In [171]:
import math

def loglikehood(sequences, PWM, f_0):
    #print(k)
    res = []
    #nb_motifs = 0
    for seq in sequences:
        liste_motifs = seeds(k, seq)
        #nb_motifs += len(liste_motifs)
        for motif in liste_motifs:
            j = 0
            score = 0
            while j < len(motif):
                lettre = motif[j]
                prob_lettre = PWM[nuc.index(lettre)][j]
                if prob_lettre == 0.0:
                    #aucune probabilité d'obtenir cette lettre
                    score = -1
                    break
                f0_lettre = f_0[lettre]
                score += math.log2(prob_lettre / f0_lettre)
                j += 1
            if score > 0 and motif not in res:
                res.append(motif)
    #print(nb_motifs)
    return res

14\.Tester l'algorithme sur vos données de chipSeq. Vous devez parcourir les séquences et extraire les motifs ayant  log-rapport de vraisemblance positive 

In [172]:
from collections import Counter
m = 5

def removeLowComplexe(motifs, minrep):
    validMotif = []
    for motif in motifs:
        frequence = Counter(motif)
        # temoin pour savoir si une lettre est repetee ou non
        temoin = True
        for lettre, value in frequence.items():
            if value >= minrep:
                temoin = False
        if temoin == True:
            validMotif.append(motif)
                
    return validMotif

def removeLowComplexePair(motifs):
    validMotif = []
    for motif in motifs:
       frequence = Counter(motif)
       if len(frequence) > 2:
           validMotif.append(motif)

    return validMotif

In [173]:

motif_mac1 = "TTTGCTCA"
PWM_mac = pwm(Mac1, k)
f_0_mac = f0_calcule(PWM_mac, k)
liste_res = loglikehood(sequences, PWM_mac, f_0_mac)
#remove motifs non interessants
liste_res = removeLowComplexe(liste_res, m)
liste_res = removeLowComplexePair(liste_res)
print(len(liste_res))
print(liste_res)
if motif_mac1 in liste_res:
    print("oui")

14
['AATACTCT', 'TTTGCTCA', 'ATTGCTCT', 'TGTACTCT', 'TTTGCTCG', 'ATTGCTCA', 'TATGCTCA', 'TGTGCTCA', 'ATTACTCA', 'AGTACTCA', 'TGTACTCG', 'AGTACTCT', 'AATGCTCT', 'ATTACTCT']
oui


### D -  Localisation de facteur de transcriptions
15\. Une fois que vous avez localisé le motif dans le donnée de chipSeq, nous allons identifier les fonctions de gènes qui sont associées aux facteurs de transcription. Pour cela, nous allons utiliser le fichier qui contient tout le génome (C_glabrata_1000bp_only.fasta), localiser le motif (vous pouvez utiliser un des algos précédent), et identifier les gènes. Ensuite nous pouvons vérifier leurs fonctions dans le fichier "C_glabrata_annotations.csv" à l'aide de ses identifiant.

In [174]:
k = 8
genome = readFasta("C_glabrata_1000bp_only.fasta")
print(len(genome))
res = {}
cpt = 0
num_seq = 0
for one_genome in genome:
    #print(len(one_genome))
    start = time.time()
    for motif in liste_res:
        pos = searchIndexTable(one_genome, k, motif)
        if pos != -1:
            if motif not in res:
                res[motif] = [num_seq]
            else:
                res[motif].append(num_seq)
    num_seq += 1

    end = time.time()
    print(end-start)
print(cpt)
print(len(res))

1347656
0.004999876022338867
0.0050013065338134766
0.004500865936279297
0.004500389099121094
0.00550079345703125
0.005001068115234375
0.0050008296966552734
0.0050008296966552734
0.0050008296966552734
0.0050008296966552734
0.005000591278076172
0.005000591278076172
0.0050013065338134766
0.004500865936279297
0.004500389099121094
0.004500389099121094
0.046508073806762695
0.005000591278076172
0.005001068115234375
0.005000114440917969
0.004500389099121094
0.004500627517700195
0.004500865936279297
0.004500627517700195
0.0050008296966552734
0.004500627517700195
0.0050008296966552734
0.005001068115234375
0.005000591278076172
0.004500865936279297
0.0050008296966552734
0.004500627517700195
0.0049991607666015625
0.0050008296966552734
0.0050008296966552734
0.004500865936279297
0.0050008296966552734
0.0045011043548583984
0.004500627517700195
0.0045011043548583984
0.005501747131347656
0.004999876022338867
0.0050013065338134766
0.006000995635986328
0.0050008296966552734
0.005501985549926758
0.00499963

In [213]:
def readProteinProduct(genome):
    sequence = []
    file = open(genome, "r")
    proteinNames = []
    for s in file:
        if s[0] == ">":
            proteinNames.append(s.strip().upper()[1:])
    return proteinNames

proteinProduct = readProteinProduct("C_glabrata_1000bp_only.fasta")


In [214]:
liste_protein_product_mac1 = []
for num_sequence in res[motif_mac1]:
    liste_protein_product_mac1.append(proteinProduct[num_sequence])

print(len(liste_protein_product_mac1))
print(liste_protein_product_mac1)

164
['CAGL0L04202G', 'CAGL0D04576G', 'CAGL0L07546G', 'CAGL0G07623G', 'CAGL0B00506G', 'CAGL0G02299G', 'CAGL0C04631G', 'CAGL0C00407G', 'CAGL0J01001G', 'CAGL0H06831G', 'CAGL0C00561G', 'CAGL0B04411G', 'CAGL0C01067G', 'CAGL0C00627G', 'CAGL0G05005G', 'CAGL0K03905G', 'CAGL0K11748G', 'CAGL0A01826G', 'CAGL0K06149G', 'CAGL0I10560G', 'CAGL0D04444G', 'CAGL0M12947G', 'CAGL0H05005G', 'CAGL0L09647G', 'CAGL0F05027G', 'CAGL0G04191G', 'CAGL0L00869G', 'CAGL0I10967G', 'CAGL0B04565G', 'CAGL0H10538G', 'CAGL0L03784G', 'CAGL0F03597G', 'CAGL0G09262G', 'CAGL0M12001G', 'CAGL0E05258G', 'CAGL0I10450G', 'CAGL0H02827G', 'CAGL0J02398G', 'CAGL0J03806G', 'CAGL0M12683G', 'CAGL0J05500G', 'CAGL0C04719G', 'CAGL0H10340G', 'CAGL0J04004G', 'CAGL0B01144G', 'CAGL0I06743G', 'CAGL0L01397G', 'CAGL0H00572G', 'CAGL0K06303G', 'CAGL0J06050G', 'CAGL0F07755G', 'CAGL0G05115G', 'CAGL0F04411G', 'CAGL0L01529G', 'CAGL0K05753G', 'CAGL0D02244G', 'CAGL0H02937G', 'CAGL0I02750G', 'CAGL0C01727G', 'CAGL0H09680G', 'CAGL0K08316G', 'CAGL0G09273G', 'CA

In [217]:
import csv
csv_bien_foutu = []
proteinNames = []

with open('C_glabrata_annotations.csv', newline='') as csvfile:
    annot = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in annot:
        ligne = ', '.join(row).upper()
        csv_bien_foutu.append(ligne.split('"'))
for csv_ligne in csv_bien_foutu[1:]:
    if csv_ligne[7] in liste_protein_names_mac1:
        proteinNames.append(csv_ligne[11])
print(len(proteinNames))
print(proteinNames)


159
['UNCHARACTERIZED, PROTEIN, CAGL0K13024G', 'UNCHARACTERIZED, PROTEIN, CAGL0A01133G', 'UNCHARACTERIZED, PROTEIN, CAGL0A01826G', 'UNCHARACTERIZED, PROTEIN, CAGL0I00506G', 'UNCHARACTERIZED, PROTEIN, CAGL0I02464G', 'UNCHARACTERIZED, PROTEIN, CAGL0I02596G', 'UNCHARACTERIZED, PROTEIN, CAGL0I02750G', 'UNCHARACTERIZED, PROTEIN, CAGL0I04092G', 'UNCHARACTERIZED, PROTEIN, CAGL0I04378G', 'UNCHARACTERIZED, PROTEIN, CAGL0I04422G', 'UNCHARACTERIZED, PROTEIN, CAGL0I04444G', 'UNCHARACTERIZED, PROTEIN, CAGL0B00506G', 'UNCHARACTERIZED, PROTEIN, CAGL0B01144G', 'UNCHARACTERIZED, PROTEIN, CAGL0I05258G', 'UNCHARACTERIZED, PROTEIN, CAGL0I05324G', 'UNCHARACTERIZED, PROTEIN, CAGL0I05478G', 'MITOCHONDRIAL, 54S, RIBOSOMAL, PROTEIN, YML9', 'UNCHARACTERIZED, PROTEIN, CAGL0B02211G', 'UNCHARACTERIZED, PROTEIN, CAGL0F00209G', 'UNCHARACTERIZED, PROTEIN, CAGL0F00979G', 'UNCHARACTERIZED, PROTEIN, CAGL0I06380G', 'UNCHARACTERIZED, PROTEIN, CAGL0I06644G', 'UNCHARACTERIZED, PROTEIN, CAGL0I06743G', 'UNCHARACTERIZED, PROTE